Install Pyspark

In [ ]:
!pip install pyspark

Request for dataset

In [ ]:
!wget https://raw.githubusercontent.com/prasadpatil99/Recommendation-System/master/ratings.csv

Create Pysprark Session & Import Dataset

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [ ]:
data1 = spark.read.csv('ratings.csv',inferSchema=True,header=True)
features_drop = ['timestamp']
data = data1.drop(*features_drop)

In [ ]:
data.printSchema()

In [ ]:
data.show()

In [ ]:
data.describe().show()

Split Dataset into train & test

In [ ]:
(train, test) = data.randomSplit([0.7, 0.2])

Training Model

In [ ]:
als = ALS(maxIter=4, regParam=0.15, userCol="userId", 
          itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(train)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

Evaluate Predictions via Root - Mean - Square 

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Predictions for a multiple users

In [ ]:
predict = test.filter(test['userId']>500).select(['movieId','userId'])

In [ ]:
reccomendations = model.transform(predict)

In [ ]:
reccomendations.orderBy('prediction',ascending=False).show()